# Identifying Bias and Correlations Guide


## What you'll do

You will learn to identify bias and correlations, understand their impact on your data, and mitigate them in the future.


## What you'll learn

- You'll learn how to use DataEval's `balance`, `diversity` and `parity` functions to identify bias and correlations present in a dataset.
- You'll be able to create a workflow using DataEval for identifying bias.


## What you'll need

- Environment Requirements
  - `torchvision`
  - `matplotlib`

---


## Introduction

Identifying any biases or correlations present in a dataset is essential to accurately interpreting your model's performance and its ability to generalize to new data.
A common cause of poor generalization is shortcut learning &mdash; where a model uses secondary or background information to make predictions &mdash; which is enabled or exacerbated by dataset sampling biases.
Understanding biases or correlations present in your dataset is a key component to creating meaningful data splits.
Bias in data can lead to misleading conclusions and poor model performance on operational data.
There are many different [types of bias](https://arxiv.org/abs/1908.09635).
A few of these biases occur during data collection, others occur during dataset development, others occur during model development, while others are a result of the user.
This guide does not focus on elminiating all bias. It focuses on identifying the bias that can be found when developing a dataset.

DataEval has three dedicated methods for identifying and understanding the bias or correlations that may be present in a dataset, the `balance`, `diversity` and `parity` functions.
The balance function measures correlational relationships between metadata factors and classes by calculating the mutual information between the metadata factors and the labels.
The diversity function measures the evenness or uniformity of the sampling of metadata factors over a dataset using the inverse Simpson index or Shannon index.
The parity function measures the relationship between metadata factors and classes using a chi-squared test.

These techniques help ensure that when you split the data for your projects, you minimize things like shortcut learning and leakage between training and testing sets.


### Importing the necessary libraries

You'll begin by importing the necessary libraries to walk through this guide.


In [ ]:
try:
    import google.colab  # noqa: F401

    # specify the version of DataEval (==X.XX.X) for versions other than the latest
    %pip install -q dataeval[torch]
except Exception:
    pass

In [ ]:
# You will need matplotlib for visualing your results and numpy to be able to handle the data.
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# You will only use torchvision to load in the dataset.
# If you already have the data stored on your computer in a numpy friendly manner,
# then feel free to load it directly into numpy arrays.
import torchvision.transforms.v2 as v2
from torchvision import datasets

# Load the classes from DataEval that are helpful for bias
from dataeval.metrics.bias import balance, diversity, parity

Below is code to help create the visualizations for the bias metrics.


In [2]:
# Creating a helper plotting function
def heatmap(
    data,
    row_labels,
    col_labels,
    xlabel="",
    ylabel="",
    top=True,
    cbarlabel="",
    valfmt=None,
    textkw=None,
    **kwargs,
):
    fig, ax = plt.subplots(figsize=(10, 10))

    # Plot the heatmap
    im = ax.imshow(data, vmin=0, vmax=1.0, **kwargs)

    # Create colorbar
    cbar = fig.colorbar(im, shrink=0.5)
    cbar.set_ticks([0.0, 0.25, 0.5, 0.75, 1.0])
    cbar.set_ticklabels(["0.0", "0.25", "0.5", "0.75", "1.0"])
    cbar.set_label(cbarlabel, loc="center")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    if top:
        ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
        # Rotate the tick labels and set their alignment.
        plt.setp(ax.get_xticklabels(), rotation=-30, ha="right", rotation_mode="anchor")
    else:
        ax.tick_params(top=False, bottom=True, labeltop=False, labelbottom=True)
        # Rotate the tick labels and set their alignment.
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1] + 1) - 0.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0] + 1) - 0.5, minor=True)
    ax.grid(which="minor", color="w", linestyle="-", linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    if xlabel:
        ax.set_xlabel(xlabel)
    if ylabel:
        ax.set_ylabel(ylabel)

    # Get the formatter in case a string is supplied
    if valfmt and isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)  # type: ignore
    else:
        valfmt = matplotlib.ticker.FuncFormatter(func)  # type: ignore

    # Normalize the threshold to the images color range.
    threshold = im.norm(1.0) / 2.0

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = {"horizontalalignment": "center", "verticalalignment": "center"}
    if textkw:
        kw.update(textkw)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    textcolors = ("white", "black")
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)  # type: ignore
            texts.append(text)

    fig.tight_layout()
    plt.show()


# Function to define how the text is displayed in the heatmap
def func(x, pos):
    return f"{x:.2f}".replace("0.00", "0").replace("0.", ".").replace("nan", "")

## Step 1: Load the Data

You are going to work with the PASCAL VOC 2011 dataset.
This dataset is a small curated dataset that was used for a computer vision competition.
The images were used for classification, object detection, and segmentation.
This dataset was chosen because it has multiple classes and a variety of images and metadata.

If this data is already on your computer you can change the file location from `"./data"` to wherever the data is stored.
Remember to also change the download value from `True` to `False`.

For the sake of ensuring that this tutorial runs quickly on most computers, you are going to analyze only the training dataset, which is a little under 6000 images.


In [ ]:
# Download the data and then load it as a torch Tensor.
to_tensor = v2.ToImage()
ds = datasets.VOCDetection(root="./data", year="2011", image_set="train", download=True, transform=to_tensor)

In [ ]:
# Verify the size of the loaded dataset
len(ds)

Before moving on, verify that the above code cell printed out 5717 for the size of the [dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2011/dbstats.html).

This ensures that everything is working as needed for the tutorial.


## Step 2: Structure the Metadata

As this data was used for a computer vision competition, it will most likely have very few issues, but it is always worth it to check.
Many of the large webscraped datasets available for use do contain image issues.
Verifying in the beginning that you have a high quality dataset is always easier than finding out later that you trained a model on a dataset with erroneous images or a set of splits with leakage.

This guide focuses on the labels and metadata for the images, rather than the images themselves, so you will load in the labels and metadata.
However, there is no standard for metadata associated with images.
Thus, you will load the metadata associated with the first image to explore it's metadata structure and determine exactly what is contained where in the metadata.
This way you can extract all of the metadata into their associated groupings.

For this dataset, the second element of a dataset item contains the metadata in a dictionary.


In [ ]:
# Check the label structure
ds[0][1]

Above shows that the metadata comes through as a nested dictionary.  
You will restructure the above metadata to be a single-layer dictionary of lists.
A few of the DataEval functions expect the labels representing the classes to be a separate list.

To help you understand what categories are good ones to keep and what categories are not, below will walk you through the current metadata categories.
This raw metadata dictionary contains 23 different dictionary keys, but not all of them contain useful information.

Metadata keys

- annotation
- folder
- filename
- source
- database
- annotation
- image
- size
- width
- height
- depth
- segmented
- object
- name
- pose
- truncated
- occluded
- bndbox
- xmin
- ymin
- xmax
- ymax
- difficult

Now you need to determine which keys contain useful information and which keys do not. Below provides context with why a key is useful or not:

- Outer _annotation_ category: **not useful** because its value is another dictionary.
- _folder_: **not useful** because it is arbitrary and lacks information about the dataset. (These values could change from computer to computer.)
- _filename_: **not useful** because it is arbitrary and lacks information about the dataset.
- _source_: **not useful** because its value is another dictionary.
- _database_: **useful** because its value contains information about the images.
- _annotation_: **useful** because its value contains information abou the images.
- _image_: **useful** because its value contains information abou the images.
- _size_: **not useful** because its value is another dictionary.
- _width_: **useful** because its value contains information abou the images.
- _height_: **useful** because its value contains information abou the images.
- _depth_: **useful** because its value contains information abou the images.
- _segmented_: **useful** because its value contains information abou the images.
- _object_: **not useful** because its value is another dictionary.
- _name_: **useful** because its value contains the class information.
- _pose_: **useful** because its value contains information abou the images.
- _truncated_: **useful** because its value contains information abou the images.
- _occluded_: **useful** because its value contains information abou the images.
- _bndbox_: **not useful** because its value is another dictionary.
- _xmin_: **useful** because its value contains information abou the images.
- _ymin_: **useful** because its value contains information abou the images.
- _xmax_: **useful** because its value contains information abou the images.
- _ymax_: **useful** because its value contains information abou the images.
- _difficult_: **useful** because its value contains information abou the images.

There is one additional distinction to make from the keys, _database_ and _annotation_ are almost identical keys as they represent the same information and therefore you only need one of them.
Now, run through all of the metadata to create the dictionary and labels list.


In [6]:
metadata_dict = {
    "database": [],
    "image": [],
    "width": [],
    "height": [],
    "channels": [],
    "segmented": [],
    "pose": [],
    "truncated": [],
    "occluded": [],
    "xmin": [],
    "ymin": [],
    "xmax": [],
    "ymax": [],
    "difficult": [],
}

class_labels = []

for data in ds:
    base = data[1]["annotation"]
    objects = base["object"]
    for obj in objects:
        # Append each value to the corresponding list in the metadata_dict
        metadata_dict["database"].append(base["source"]["database"])
        metadata_dict["image"].append(base["source"]["image"])
        metadata_dict["width"].append(int(base["size"]["width"]))
        metadata_dict["height"].append(int(base["size"]["height"]))
        metadata_dict["channels"].append(int(base["size"]["depth"]))
        metadata_dict["segmented"].append(int(base["segmented"]))
        metadata_dict["pose"].append(obj["pose"])
        metadata_dict["truncated"].append(int(obj["truncated"]))
        metadata_dict["occluded"].append(int(obj["occluded"]))
        metadata_dict["xmin"].append(int(obj["bndbox"]["xmin"]))
        metadata_dict["ymin"].append(int(obj["bndbox"]["ymin"]))
        metadata_dict["xmax"].append(int(obj["bndbox"]["xmax"]))
        metadata_dict["ymax"].append(int(obj["bndbox"]["ymax"]))
        metadata_dict["difficult"].append(int(obj["difficult"]))
        class_labels.append(obj["name"])

In [ ]:
# Verify that the lengths match
print(len(class_labels), len(metadata_dict["database"]))

Also, the `parity` function does not work with continuous data like _xmin_, so you have to discretize it into bins.
An easy way to do this is to use numpy's histogram function with "auto" bins.
However, that method can still result in bins with only a few values in them.

The parity function relies on a [contingency table](https://en.wikipedia.org/wiki/Contingency_table) and recommends at least 5 samples per table slot.
In order to achieve this recommended 5 samples per table slot in the contingency, the continuous data needs to be binned with a relatively large number of samples per bin.
To achieve this, you will use bins with at least 1500 samples in it. The code below implements this binning of the continuous data.

It's not always possible to meet the 5 sample minimum, and while this will not prevent the parity function from running,
it will skew the results for the specific factor towards having bias and it will issue a warning informing you of the value-label combination that did not meet the recommended 5 samples.
This means that the p-values are smaller than they might be if you met the recommended 5 samples.


In [8]:
min_count = 1500

for continuous_variable in ["width", "height", "channels", "xmin", "ymin", "xmax", "ymax"]:
    counts, bin_edges = np.histogram(metadata_dict[continuous_variable], bins="auto")

    if counts.min() < min_count:
        new_counts = []
        new_bin_edges = [bin_edges[0]]

        current_count = 0
        for i in range(len(counts)):
            current_count += counts[i]

            if current_count >= min_count:
                new_counts.append(current_count)
                new_bin_edges.append(bin_edges[i + 1])
                current_count = 0

        if current_count > 0:
            new_counts[-1] += current_count

        if counts[-1] < min_count:
            new_bin_edges[-1] = bin_edges[-1] + 1

        # Output the adjusted histogram
        new_counts = np.array(new_counts)
        new_bin_edges = np.array(new_bin_edges)

        discretized_variable = np.digitize(metadata_dict[continuous_variable], bins=new_bin_edges, right=False)

    else:
        discretized_variable = np.digitize(metadata_dict[continuous_variable], bins=bin_edges, right=False)

    metadata_dict[continuous_variable] = list(discretized_variable)

Now that you have translated the metadata into a single-layer dictionary and discretized the continuous variables, a few of the DataEval bias metrics require the metadata to be in the form of a list of dictionaries.
Also, the class labels list needs to be translated from label strings to numerical format.


In [ ]:
keys = metadata_dict.keys()
metadata = [dict(zip(keys, values)) for values in zip(*metadata_dict.values())]
print(len(metadata))

class_list, numerical_labels = np.unique(class_labels, return_inverse=True)
print(len(numerical_labels))

As mentioned above, there are many different forms of bias.
However, not all forms of bias directly affect the dataset and in order to address the biases that affect datasets, you have to make a few assumptions:

1. All desired classes are present.
2. All available metadata is provided.
3. The metadata has been recorded correctly.

If any of the above assumptions are violated, then the analysis below will be inaccurate. \
When using your own data, you should verify the above assumptions.

Now, you can move on with identifying any bias that may be present in the dataset.


DataEval contains 3 main functions for detecting bias and correlations in a dataset:

- balance
- diversity
- parity

You will use each function to test the dataset.


## Step 3: Checking Dataset Balance


The `balance` function measures correlational relationships between metadata factors and classes in a dataset.
It analyzes the metadata factors against both the classes and other factors to identify relationships.

The results can be retrieved using the balance and factors keys from the output.


In [10]:
bal = balance(list(numerical_labels), metadata)

The information provided by the balance function may be visually understood with a heat map.
To do this you will combine the balance and factors keys from the result into a single heat map.
The balance key is the analysis between metadata factors and class, while the factors key is the analysis between all pairs of metadata factors.


In [11]:
# Combine balance and factors results
data = np.concatenate([bal.balance[np.newaxis, 1:], bal.factors], axis=0)
# Create a mask for the upper triangle of the symmetrical array, ignoring the diagonal
mask = np.triu(data + 1, k=0) < 1
# Finalize the data for the plot, last row is last factor x last factor so it gets dropped
heat_data = np.where(mask, np.nan, data)[:-1]
# Creating label array for heat map axes
heat_labels = ["class"] + list(metadata_dict.keys())

In [ ]:
heatmap(heat_data, heat_labels[:-1], heat_labels[1:], cbarlabel="Normalized Mutual Information")

The heatmap shows that the greatest correlations are in the bounding box locations (_xmin_ with _xmax_ and _ymin_ with _ymax_) and the image dimensions (_height_ and _width_).
Also the _ymax_ of the bounding box location is correlated with the _height_ of the image.
It is not surprising that _height_ and _width_ have correlation since many of the images are similarly sized.
The correlations between _xmin_ and _xmax_ and between _ymin_ and _ymax_ suggests that there is repetition in bounding box width and height across the objects.
However, the fact that _pose_ has a value near 0.10 with _class_ means that a few of the classes have specific poses across a fair percentage of the images for that class.
An example of this would be most _pottedplant_ images having the same _pose_ value.

In addition to analyzing class and other factors, the balance function also analyzes metadata factors with individual classes to identify relationships between only one class and secondary factors.
Again, a heatmap is used for visualizing the classwise results.
These results can be retrieved using the classwise key from the output.


In [ ]:
heatmap(
    bal.classwise,
    class_list,
    heat_labels,
    xlabel="Factors",
    ylabel="Class",
    top=False,
    cbarlabel="Normalized Mutual Information",
)

The classwise heatmap shows that factors other than _class_ do not have any significant correlation with a specific class.
Classwise balance shows correlation of individual classes with all class labels, indicating relative class imbalance.
In this case the _person_ class is over-represented relative to most other classes.
This means that a model might learn a bias towards the _person_ class label due to its frequency in the training set, which becomes a problem if the test/operational dataset doesn't have the same imbalance.


## Step 4: Checking Dataset Diversity


The `diversity` function measures the evenness or uniformity of the sampling of metadata factors over a dataset.
Values near 1 indicate uniform sampling, while values near 0 indicate imbalanced sampling, e.g. all values taking a single value.

The results can be retrieved using the diversity_index key from the output.


In [14]:
div = diversity(list(numerical_labels), metadata)

Again, it's often easiest to see the differences between the different factors when visualizing them.
This time you will use a box plot as there is only a factor-class analysis, no factor-factor analysis.


In [ ]:
fig, ax = plt.subplots()
ax.bar(heat_labels, div.diversity_index)
ax.set_xlabel("Factors")
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
plt.show()

When analyzing a diversity plot, there are two key values to keep in mind, 1 and 0. A 1 represents uniform sampling and an unbiased factor, while a 0 represents a constant value for all samples.
Factors with a diversity value of 0 do not provide insight as to whether you have adequately sampled the data space, for more information see the [Diversity](../concepts/Diversity.md) concept page. Diversity values near 1 indicate unbiased factors while values below 0.5 indicate basis in the factor.

In the results above, the metadata factors _image_ and _channels_ contain only a single value and do not contribute to our bias analysis.
The factors _truncated_ and _occluded_ have values near 1, meaning that there is relatively little or no bias in these factors.
The categories of most interest are those that are between 0.5 and 0.2 because this region represents skewed value distributions for the factor.
The following factors fall into this category:

- _class_
- _width_
- _height_
- _segmented_
- _difficult_

These factors contain bias that should be addressed either by adding or removing data to even out the sampling.
For instance, the _class_ factor highlights that there is unevenness in the number of data points per class.

In addition to analyzing class, the diversity function also analyzes metadata factors with individual classes to assess uniformity of metadata factors within a class.
As above, a heatmap is used for visualizing the classwise results.
These results can be retrieved using the classwise key from the output.


In [ ]:
heatmap(
    div.classwise,
    class_list,
    list(metadata_dict.keys()),
    xlabel="Factors",
    ylabel="Class",
    top=False,
    cbarlabel="Normalized Simpson Index",
)

These results expand the above results on a per class basis.
Things to look for here are large variances for a given factor across the different classes.
For example, _pose_ has values ranging from 0.01 to 0.84, which means that a few classes have almost uniform selection of the different _pose_ values while other classes essentially only have one _pose_ value.
Should classes have different selections of _pose_ value? Yes, one would not expect the _diningtable_ class to have a _pose_ direction, while the _person_ class should have multiple _pose_ directions.
What needs to be further investigated are things like whether the _sofa_ class should have a _pose_ direction, because a diversity value of 0.4 means that a few of the images do while others do not.
Also, the _cat_ class has a low score signifying that most of the images fall into one or two categories rather than being spread even across the categories.
This highlights an error in the data collection process &mdash; the value was not specified for most _cat_ images and therefore defaulted to "Unspecified".
An alternative error would be a dataset in which the _cat_ images have most cats facing a specific direction, which would require additional data to overcome the bias,
but that is not the case for this dataset. It has plenty of cats facing each direction, but only a few of them contain a _pose_ value.


## Step 5: Checking Dataset Parity


The `parity` function measures the relationship between metadata factors and classes using a chi-squared test.
A high score with a low p-value suggests that a metadata factor is strongly correlated with a class label.

Parity requires the metadata format to be a dictionary of lists with the class labels included and the class labels are required to be in numerical format (no strings).

The results can be retrieved using the score and p-value keys from the output.


In [17]:
metadata_dict["class"] = list(numerical_labels)

In [ ]:
par = parity(metadata_dict)

The warning above states that the metric works best when there are more than 5 samples in each value-label combination.
However, because of the large number of total samples, the difference between 1 and 5 samples does not significantly affect the results.

When evaluating the results of parity for a large number of factors, it may be easier to understand the results by presenting the score and p-value for a given factor together.
The code below restructures the result to this structure.


In [ ]:
for key, score, value in zip(metadata_dict.keys(), par.score, par.p_value):
    print(f"{key} - {score} : {value}")

According to the results, all metadata are correlated with _class_ labels, except for _image_ and _channels_.
However, parity is based on the idea of an expected frequency and how the observed differs from what is expected.
The expected frequencies are determined by sums of the values for each metadata category.

This function works best when the expected frequencies for a given factor for each individual class are known a priori.
So for the case above, the expected frequency for the _pose_ metadata category shouldn't be the same for all classes.
_Diningtable_, _pottedplant_, and _bottle_ should only have a single value for _pose_ which automatically throws off the metric because not all of the classes have an identical expected frequency for _pose_.


## Conclusion


Having analyzed the dataset for bias with multiple metrics, the concluding answer is this dataset has bias.
Training a model on this dataset has the potential to learn shortcuts and underperform on operational data if the biases are not representative of biases in the operational dataset.

The metadata categories identified by the `balance`, `diversity` and `parity` functions contain issues such as imbalanced classes and imbalanced parameters per class.
DataEval isn't able to tell you exactly why they are imbalanced, but it highlights the categories that you need to check.

As you can see, the DataEval methods are here to help you gain a deep understanding of your dataset and all of its strengths and limitations.
It is designed to help you create representative and reliable datasets.

Good luck with your data!

---


## What's Next

In addition to identifying bias and correlations in a dataset, DataEval offers additional tutorials to help you learn about dataset analysis:

- To clean a dataset use the [Data Cleaning Guide](EDA_Part1.ipynb).
- To identify coverage gaps and outliers use the [Assessing the Data Space Guide](EDA_Part2.ipynb).
- To monitor data for shifts during operation use the [Data Monitoring Guide](Data_Monitoring.ipynb).

To learn more about the balance, diversity and parity functions, see the [Balance](../concepts/Balance.md), [Diversity](../concepts/Diversity.md) and [Parity](../concepts/Parity.md) concept pages.

## On your own

Once you are familiar with DataEval and dataset analysis, you will want to run this analysis on your own dataset.
When you do, make sure that you analyze all of your data and not just the training set.
